In [13]:


!pip install lingualytics

ERROR: Could not find a version that satisfies the requirement sentence_transformer (from versions: none)
ERROR: No matching distribution found for sentence_transformer
     |████████████████████████████████| 2.8 MB 34.7 MB/s 
     |████████████████████████████████| 235 kB 48.9 MB/s 
     |████████████████████████████████| 1.5 MB 46.2 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 3.3 MB 47.0 MB/s 
     |████████████████████████████████| 636 kB 47.8 MB/s 
     |████████████████████████████████| 895 kB 46.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [15]:
!pip install sentence_transformers

     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 1.2 MB 47.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=b78d579acabf7ac7eda80243ba9cd60922b3871cd4f0c28692749419c00316df
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers


In [16]:
import pandas as pd
import numpy as np
from lingualytics.preprocessing import remove_lessthan, remove_punctuation, remove_stopwords
from lingualytics.stopwords import en_stopwords
from texthero.preprocessing import remove_digits
from sentence_transformers import SentenceTransformer
from torch.nn import CosineSimilarity
import torch 
import pickle


In [17]:
df = pd.read_csv("/content/faq.csv")
pd.set_option('display.max_colwidth',None)

In [18]:
df

,Q,A
0,What is kandi?,"kandi (pronounced kan�dee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items."
1,Have feedback or want to know more?,"We are a passionate set of application focused techies. We�d love to hear from you on your feedback, questions, and any other comments.\r\nDirect Message us on Twitter Message @OpenWeaverInc\r\nYou can email us at kandi.support@openweaver.com\r\nJoin our Discord community here"
2,What components does kandi cover?,kandi helps you select software components across:\r\nPackages from all package managers and repositories\r\nSource Code across all major code repositories\r\nCloud Functions and APIs across all hyperscale cloud providers
3,How do I use kandi?,"kandi provides two simplified experiences to help you choose the right software component to accelerate your application development:\r\n\r\n1. Search\r\nYou can search for the component using natural language to describe your functional and technical requirements, and kandi gets to work by matching these over 430 million knowledge items to show you a shortlist.\r\nYou can further filter them or refine your query and pick your chosen ones based on scores available on the component listing page.\r\nClick on the components from the list to review detailed insights such as support, quality, security, and a reference guide covering code snippets, community discussions from the provider, and popular channels.\r\nThe component listing and detailed insights page have links to the software component home. Some software components may have multiple providers, and you can access all the links.\r\n\r\n2. Explore\r\nYou can Explore kandi curated sections across Popular Collections, Hot Tech, and Industry Domains from the Home Page or the Explore Page. These sections list the popular components among your peers, have functional relevance, and positive security, quality, and support scores in the respective areas.\r\nYou can browse these sections to get industry insights.\r\nYou can further filter them and pick your chosen ones based on scores available on the component listing page.\r\nClick on the components from the list to review detailed insights such as support, quality, security, and a reference guide covering code snippets, community discussions from the provider, and popular channels.\r\nThe component listing and detailed insights page have links to the software component home. Some software components may have multiple providers, and you can access all the links."
4,How do I shortlist components on kandi?,"You can use the below filters to shortlist components based on your architectural preferences:\r\n\r\nLanguages � This is an expanding list of languages chosen by popularity amongst kandi users.\r\nLicenses � Licenses are grouped by:\r\n\r\nOSS License families, covering Permissive, Weak Copyleft, and Strong Copyleft.\r\nProprietary license category covering the emerging cloud licenses as well as As-a-Service contracts.\r\nNo License indicates that the respective repository does not have the license file declared as per the repository manager�s standard. They could still have a license file declared in a different format or section. Components without a license have all rights reserved, and you may not be able to use them. Hence kandi alerts you when a valid license file is not found.\r\n\r\nSupport � High support indicates a thriving ecosystem across the author and users, that will help you implement with relative ease.\r\nComponent Types � Component Types are grouped by:\r\n\r\nLibraries from package managers and repositories that can be readily installed.\r\nSource Code that may or may not be associated with a package and are from code repositories.\r\nCloud Functions and APIs that are provided As-a-Service from cloud providers.\r\n\r\nSources � This is an expanding list of software component sources chosen by popularity amongst kandi

In [19]:
df['procd_Q'] = df['Q'].pipe(remove_digits).pipe(remove_punctuation)

In [20]:
df

,Q,A,procd_Q
0,What is kandi?,"kandi (pronounced kan�dee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.",What is kandi
1,Have feedback or want to know more?,"We are a passionate set of application focused techies. We�d love to hear from you on your feedback, questions, and any other comments.\r\nDirect Message us on Twitter Message @OpenWeaverInc\r\nYou can email us at kandi.support@openweaver.com\r\nJoin our Discord community here",Have feedback or want to know more
2,What components does kandi cover?,kandi helps you select software components across:\r\nPackages from all package managers and repositories\r\nSource Code across all major code repositories\r\nCloud Functions and APIs across all hyperscale cloud providers,What components does kandi cover
3,How do I use kandi?,"kandi provides two simplified experiences to help you choose the right software component to accelerate your application development:\r\n\r\n1. Search\r\nYou can search for the component using natural language to describe your functional and technical requirements, and kandi gets to work by matching these over 430 million knowledge items to show you a shortlist.\r\nYou can further filter them or refine your query and pick your chosen ones based on scores available on the component listing page.\r\nClick on the components from the list to review detailed insights such as support, quality, security, and a reference guide covering code snippets, community discussions from the provider, and popular channels.\r\nThe component listing and detailed insights page have links to the software component home. Some software components may have multiple providers, and you can access all the links.\r\n\r\n2. Explore\r\nYou can Explore kandi curated sections across Popular Collections, Hot Tech, and Industry Domains from the Home Page or the Explore Page. These sections list the popular components among your peers, have functional relevance, and positive security, quality, and support scores in the respective areas.\r\nYou can browse these sections to get industry insights.\r\nYou can further filter them and pick your chosen ones based on scores available on the component listing page.\r\nClick on the components from the list to review detailed insights such as support, quality, security, and a reference guide covering code snippets, community discussions from the provider, and popular channels.\r\nThe component listing and detailed insights page have links to the software component home. Some software components may have multiple providers, and you can access all the links.",How do I use kandi
4,How do I shortlist components on kandi?,"You can use the below filters to shortlist components based on your architectural preferences:\r\n\r\nLanguages � This is an expanding list of languages chosen by popularity amongst kandi users.\r\nLicenses � Licenses are grouped by:\r\n\r\nOSS License families, covering Permissive, Weak Copyleft, and Strong Copyleft.\r\nProprietary license category covering the emerging cloud licenses as well as As-a-Service contracts.\r\nNo License indicates that the respective repository does not have the license file declared as per the repository manager�s standard. They could still have a license file declared in a different format or section. Components without a license have all rights reserved, and you may not be able to use them. Hence kandi alerts you when a valid license file is not found.\r\n\r\nSupport � High support indicates a thriving ecosystem across the author and users, that will help you implement with relative ease.\r\nComponent Types � Component Types are grouped by:\r\n\r\nLibraries from package managers and repositories that can be readily installed.\r\nSource Code that may or may not be associated with a package and are from code repositories.\r\nCloud Functions and APIs that are provided As-a-Service from cloud providers

In [21]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
model_path = "/content/model_va.pickle"
qembs = model.encode(df["procd_Q"])
with open(model_path,"wb")as file:
  pickle.dump(qembs,file)

In [23]:
with open("/content/model_va.pickle","rb") as file:
  qembs = pickle.load(file)

In [29]:
def pred(user_q):
  df_q = pd.DataFrame([user_q],columns=["user_q"])
  df_q["clean_user_q"] = df_q["user_q"].pipe(remove_digits).pipe(remove_punctuation)
  user_q_emb = model.encode(df_q["clean_user_q"])
  cos_sim = CosineSimilarity()
  q_idx = np.argmax(cos_sim(torch.from_numpy(user_q_emb),torch.from_numpy(qembs)))
  print(df["A"[q_idx.item()]])

In [31]:
usr_query = input("your doubt:")
pred(usr_query)

your doubt:who is kandi
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [32]:
while True:
    usr_q = input("Ask a query:")
    if usr_q == "exit":
        break
    else:
        print("Answer: ", pred(usr_q))
    print("-----------------")

Ask a query:kandi
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

KeyboardInterrupt: ignored